In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/open-problems-single-cell-perturbations/multiome_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/multiome_obs_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/sample_submission.csv
/kaggle/input/open-problems-single-cell-perturbations/adata_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/multiome_var_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/adata_obs_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/id_map.csv
/kaggle/input/open-problems-single-cell-perturbations/de_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/adata_excluded_ids.csv
/kaggle/input/tf-features/Mlmtf_median_output.csv
/kaggle/input/pathway-activity-inference-features/Mlm_median_output - p.csv


Install the necessary package

In [2]:
!pip install rdkit
!pip install pubchempy

  Obtaining dependency information for rdkit from https://files.pythonhosted.org/packages/04/0c/bf1325c3232230d6d5bf9df985c6fd01b121da618e88ccf98a70428fe153/rdkit-2023.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 25.8 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=fcf8a57bed21c57f3a36203148baeb8bd1d127f81dc6100c66d17e09a9bc9a80
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


Import the required library

In [3]:
import warnings 
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.utils.validation import check_is_fitted
from rdkit.Chem import rdFingerprintGenerator
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors  
from typing import List, Set, Dict, Tuple, Optional, Union
from rdkit.Chem import MACCSkeys
%matplotlib inline

Read the training gene expression data

In [4]:
de_train = pd.read_parquet(r'/kaggle/input/open-problems-single-cell-perturbations/de_train.parquet')
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [5]:
# split_index = 5
# de_train_2 = de_train.iloc[:,split_index:]
# de_train_2.head()

In [6]:
# # Step 1: Gene Co-Expression Network Construction
# correlation_matrix = de_train_2.corr()  # Calculate pairwise correlations
# threshold = 0.7  # Define a correlation threshold
# adjacency_matrix = np.where(np.abs(correlation_matrix) > threshold, 1, 0)  # Thresholding

In [7]:
# import networkx as nx
# # Step 2: Calculate Network Properties
# coexp_network = nx.Graph(adjacency_matrix)


In [8]:
# coexp_network

In [9]:
# degree_centrality = nx.degree_centrality(coexp_network)
# clustering_coefficient = nx.clustering(coexp_network)
# betweenness_centrality = nx.betweenness_centrality(coexp_network)

# # Print or visualize the results
# print(f"Degree Centrality: {degree_centrality}")
# print(f"Clustering Coefficient: {clustering_coefficient}")
# print(f"Betweenness Centrality: {betweenness_centrality}")

Define the target variable

In [10]:
genes = de_train.columns[5:]

Added extra feature generated from decoupleR package

In [11]:
new_feature = pd.read_csv(r"/kaggle/input/tf-features/Mlmtf_median_output.csv")
new_feature= new_feature.drop(['Unnamed: 0', 'cell_type', 'sm_name'], axis = 1)


The above function uses the PubChemPy library to retrieve the SMILES notation for a list of chemical compound names. 
SMILES (Simplified Molecular Input Line Entry System) is a way to represent the structure of chemical molecules using simple text strings.

In [12]:

import pubchempy as pcp  # Import the PubChemPy library for chemical compound information
import pandas as pd  # Import the Pandas library for data handling
import re  # Import the regular expressions library for text pattern matching
from urllib.request import urlopen
from urllib.parse import quote

# Define a function to retrieve SMILES notation for given compound names
def get_smiles_for_compounds(compound_names):
    # Create an empty list to store the results
    results_list = []

    # Iterate through each compound name in the provided list
    for compound_name in compound_names:
        # Check if the compound name contains a semicolon, and if so, split it and use the first part
        if re.search(r'\;', compound_name):
            compound_name = compound_name.split(';')[0]

        # Search for the compound by name in the PubChem database
        results = pcp.get_compounds(compound_name, 'name')

        if results:
            # Get the first result (most relevant match)
            compound = results[0]

            # Check if SMILES information is available for the compound
            if 'canonical_smiles' in compound.to_dict():
                # Extract the SMILES notation
                smiles = compound.to_dict()['canonical_smiles']

                # Append the compound name and SMILES notation to the results list
                results_list.append({'Compound Name': compound_name, 'SMILES Notation': smiles})
            else:
                # Print a message if SMILES information is not available for the compound
                print(f"SMILES information not available for '{compound_name}'.")
        else:
            # Print a message if no compound is found with the given name in PubChem
            print(f"No compound found with the name '{compound_name}' in PubChem.")

    # Create a DataFrame from the list of results
    df = pd.DataFrame(results_list)

    # Return the DataFrame containing the compound names and SMILES notations
    return df


The above function utilizes the RDKit library to calculate various molecular descriptors and fingerprints for a list of chemical compounds represented by SMILES strings.

In [13]:
# Import necessary libraries from the RDKit and Pandas
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdFingerprintGenerator
import pandas as pd
import numpy as np

# Function to calculate various molecular descriptors for a list of SMILES strings
def calculate_all_descriptors(smiles_list):
    results = []

    for smiles in smiles_list:
        # Convert the SMILES string to an RDKit molecule object
        mol = Chem.MolFromSmiles(smiles)

        if mol is not None:
            descriptors = {}
            descriptors["SMILES"] = smiles

            # Calculate all available descriptors using RDKit
            for desc_name, desc_function in Descriptors.descList:
                try:
                    value = desc_function(mol)
                    descriptors[desc_name] = value
                except Exception as e:
                    # In case of an error, add an error message
                    descriptors[desc_name] = str(e)

            results.append(descriptors)
        else:
            # If the SMILES string is invalid, add an error message
            results.append({"SMILES": smiles, "Error": "Invalid SMILES"})

    return results

# Function to calculate MACCS fingerprints for a list of SMILES strings
def calculate_macss_descriptors(smiles_list):
    results = []

    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)

        if mol is not None:
            descriptors = {}
            descriptors["SMILES"] = smiles

            # Calculate MACCS fingerprints
            maccs_fp = MACCSkeys.GenMACCSKeys(mol)
            # Add MACCS fingerprint bits as descriptors
            descriptors.update({f"macss_{i}": bit for i, bit in enumerate(maccs_fp.ToBitString())})

            results.append(descriptors)
        else:
            # If the SMILES string is invalid, add an error message
            results.append({"SMILES": smiles, "Error": "Invalid SMILES"})

    # Create a DataFrame from the results
    df = pd.DataFrame(results)
    return df

# Function to compute Morgan fingerprints for a list of SMILES strings
def compute_morgan_fingerprints(df, smiles_column='SMILES', radius=2, fp_size=512, include_chirality=False):
    morgan_fp_gen = rdFingerprintGenerator.GetMorganGenerator(includeChirality=include_chirality, radius=radius, fpSize=fp_size)
    
    df_mfp = None
    
    for i in range(len(df)):
        # Convert the SMILES string to an RDKit molecule object
        mol = Chem.MolFromSmiles(df[smiles_column].iloc[i])
        # Generate Morgan fingerprint for the molecule
        fp = morgan_fp_gen.GetFingerprint(mol)
        
        if df_mfp is None:
            df_mfp = np.array(fp)
        else:
            df_mfp = np.vstack([df_mfp, np.array(fp)])
    
    # Create a DataFrame from the Morgan fingerprints
    df_mfp = pd.DataFrame(df_mfp)
    df_mfp.columns = ['MFP_' + str(i) for i in range(df_mfp.shape[1])]
    
    return df_mfp


The above function prepares the data by extracting chemical features, encoding categorical labels, and creating a dataset suitable for training a model to predict cell types based on chemical compound characteristics.

In [14]:
# Import necessary libraries
from sklearn.preprocessing import LabelEncoder

# Function to create training data with descriptors
def train_data_creation(df, smiles_col='SMILES'):
    # Initialize a LabelEncoder to encode 'cell_type' values as numbers
    label_encoder = LabelEncoder()
    df['cell_type'] = label_encoder.fit_transform(df['cell_type'])
    
    # Make a copy of the original DataFrame
    df_ = df.copy()
    
    # Extract SMILES strings from the DataFrame
    smiles_list = df_[smiles_col].to_list()
    
    # Calculate various RDKit descriptors for the SMILES strings
    rdkit_descriptor_info = calculate_all_descriptors(smiles_list)
    rdkit_descriptor_df = pd.DataFrame(rdkit_descriptor_info)
    
    # Calculate MACCS fingerprints for the SMILES strings
    morgan_descriptor_df = calculate_macss_descriptors(smiles_list)
    
    # Concatenate the RDKit descriptors and MACCS fingerprints
    df_features = pd.concat([rdkit_descriptor_df, morgan_descriptor_df], axis=1)
    
    # Fill missing values with zeros in the feature DataFrame
    df_features.fillna(0, inplace=True)
    
    # Extract the initial feature column names
    feature_cols = df_features.drop(['SMILES'], axis=1).columns.to_list()

    # Add new feature names to the feature_cols list
    #new_feature_names = ['MYC', 'TP53', 'STAT1', 'HIF1A', "FOXO3", "ZBTB4", "MLXIP", "E2F4", "HIVEP2", "SIN3A"]
    new_feature_names= ['MYC', 'TP53', 'MLXIP','FOXO3','STAT1','ZBTB4','TBPL2',"E2F4", "HIF1A", "HSF1"]
    feature_cols.extend(new_feature_names)
    
    # Concatenate the original DataFrame with the feature DataFrame
    final_df = pd.concat([df_, df_features], axis=1)
    
    return final_df, feature_cols, label_encoder

Function is designed to create test data for evaluating a model.

In [15]:
# Function to create test data with descriptors
def test_data_creation(df, label_encoder, sm_name_col='sm_name', smiles_col='SMILES'):
    # Encode 'cell_type' values using the same LabelEncoder from training data
    df['cell_type'] = label_encoder.transform(df['cell_type'])
    
    # Make a copy of the original DataFrame
    df_ = df.copy()
    
    # Add new feature names to the df_ DataFrame
    #new_feature_names = ['MYC', 'TP53', 'STAT1', 'HIF1A', "FOXO3", "ZBTB4", "MLXIP", "E2F4", "HIVEP2", "SIN3A"]
    new_feature_names= ['MYC', 'TP53', 'MLXIP','FOXO3','STAT1','ZBTB4','TBPL2',"E2F4", "HIF1A", "HSF1"]
    for new_feature in new_feature_names:
        df_[new_feature] = 0  # Add new feature columns with initial value 0
    
    # Extract SMILES strings from the DataFrame
    smiles_list = df_[smiles_col].to_list()
    
    # Calculate various RDKit descriptors for the SMILES strings
    rdkit_descriptor_info = calculate_all_descriptors(smiles_list)
    rdkit_descriptor_df = pd.DataFrame(rdkit_descriptor_info)
    
    # Calculate MACCS fingerprints for the SMILES strings
    morgan_descriptor_df = calculate_macss_descriptors(smiles_list)
    
    # Concatenate the RDKit descriptors and MACCS fingerprints
    df_features = pd.concat([rdkit_descriptor_df, morgan_descriptor_df], axis=1)
    
    # Fill missing values with zeros in the feature DataFrame
    df_features.fillna(0, inplace=True)
    
    # Extract the initial feature column names
    feature_cols = df_features.drop(['SMILES'], axis=1).columns.to_list()
    
    # Concatenate the original DataFrame with the feature DataFrame
    final_df = pd.concat([df_, df_features], axis=1)
    
    return final_df, feature_cols


In [16]:
train_df = de_train[['cell_type','sm_name','SMILES'] + genes.to_list()]
train_df.head()

,cell_type,sm_name,SMILES,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [17]:
train_df, feature_cols,label_encoder = train_data_creation(df = train_df)

In [18]:
train_df

,cell_type,sm_name,SMILES,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,...,macss_157,macss_158,macss_159,macss_160,macss_161,macss_162,macss_163,macss_164,macss_165,macss_166
0,2,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,...,0,1,0,0,1,1,1,0,1,0
1,3,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,...,0,1,0,0,1,1,1,0,1,0
2,4,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,...,0,1,0,0,1,1,1,0,1,0
3,5,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,...,0,1,0,0,1,1,1,0,1,0
4,2,Mometasone Furoate,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,...,1,0,1,1,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,5,Atorvastatin,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,-0.544709,0.282458,...,1,1,1,1,1,1,1,1,1,0
610,2,Riociguat,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,-0.455549,0.188181,0.595734,-0.100299,0.786192,0.090954,0.169523,...,1,1,1,1,1,1,1,1,1,0
611,3,Riociguat,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,0.338168,-0.109079,0.270182,-0.436586,-0.069476,-0.061539,0.002818,...,1,1,1,1,1,1,1,1,1,0
612,4,Riociguat,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,0.101138,-0.409724,-0.606292,-0.071300,-0.001789,-0.706087,-0.620919,...,1,1,1,1,1,1,1,1,1,0


Shape of training data with new features added

In [19]:

data = pd.concat([train_df, new_feature], axis=1)
data.shape

(614, 18603)

In [20]:
len(feature_cols)

387

The above function prepares data, calculates representative scores for compounds, assigns classes, and performs stratified k-fold cross-validation with the goal of evaluating model performance on different subsets of the data while ensuring a balanced distribution of classes in each fold.

In [21]:
from sklearn.model_selection import StratifiedKFold


B_M_df = de_train[de_train["cell_type"].isin(["B cells", "Myeloid cells"])].reset_index(drop=True)
mean_score = B_M_df[["sm_name"]+genes.tolist()].groupby("sm_name").agg("mean").mean(axis=1)
classes = np.digitize(mean_score.values, bins=[0.1, 0.5, 1])
cpds = mean_score.index.values
fold_arr = np.full(len(cpds), -1)
skf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
for fold, (trn_ind, val_ind) in enumerate(skf.split(classes, classes)):
    fold_arr[val_ind] = fold
fold_map = {c: f for c, f in zip(cpds, fold_arr)}
fold_to_cpds = {fold: cpds[fold_arr==fold] for fold in range(3)}

fold_arr

array([0, 0, 2, 2, 1, 1, 0, 1, 0, 0, 1, 0, 2, 1, 2, 1, 2])

Truncated SVD is used for dimensionality reduction, and the model is trained and evaluated in a cross-validated manner.

The MRRMSE is a metric used to assess the performance of the model on predicting gene expression levels.

In [22]:
from sklearn.decomposition import TruncatedSVD


temp = de_train.groupby(['cell_type']).size() > 20
validation_cell_types = temp[temp].index # 4 cell types

train_sm_names = de_train.query("cell_type == 'B cells'").sm_name.values 

features = ['cell_type', 'sm_name'] + feature_cols

def cross_val_svd(model, sel_feature, n_components=35, print_each=False):
    mrrmse_list = []
    for fold in range(3):
        #mask_va = (de_train.cell_type == val_cell_type) & ~de_train.sm_name.isin(train_sm_names)
        mask_va = de_train['cell_type'].isin(['Myeloid cells', 'B cells']) & de_train['sm_name'].isin(fold_to_cpds[fold])
        mask_tr = ~mask_va # 485 or 487 training rows
        
        train_ = data[mask_tr]
        train = train_[sel_feature]
        val_ = data[mask_va]
        val = val_[sel_feature]


        y_true = val_[genes]
        
        svd = TruncatedSVD(n_components=n_components, random_state=1)
        z_tr = svd.fit_transform(train_[genes])
        
        model.fit(train[sel_feature], z_tr)
        y_pred = svd.inverse_transform(model.predict(val[sel_feature]))
        
        mrrmse = np.sqrt(np.square(y_true - y_pred).mean(axis=1)).mean()
        if print_each:
            #print(f"# Fold {fold}: {mrrmse:5.3f} val='{val_cell_type}'")
            print(f"# Fold {fold}: {mrrmse:5.3f}")
        mrrmse_list.append(mrrmse)
    mrrmse = np.array(mrrmse_list).mean()
    print(f"# Overall {mrrmse:5.3f}")
    return mrrmse

The script evaluates the Ridge regression model's performance on the gene expression prediction task using Truncated SVD for dimensionality reduction.

Ridge regression is used to model the relationship between the selected features and gene expression levels.

The cross_val_svd function carries out the cross-validated evaluation, where the training and validation sets are defined for each fold, and the MRRMSE is calculated.

In [23]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

n_components = 35
model = Ridge(alpha= 1, fit_intercept=True, random_state=0)
cross_val_svd(model,sel_feature=feature_cols,n_components=n_components, print_each=True)

# Fold 0: 2.968
# Fold 1: 2.873
# Fold 2: 2.075
# Overall 2.639


2.638751965978183

In [24]:
id_map = pd.read_csv('/kaggle/input/open-problems-single-cell-perturbations/id_map.csv')
id_map

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)
2,2,B cells,ABT737
3,3,B cells,AMD-070 (hydrochloride)
4,4,B cells,AT 7867
...,...,...,...
250,250,Myeloid cells,Vandetanib
251,251,Myeloid cells,Vanoxerine
252,252,Myeloid cells,Vardenafil
253,253,Myeloid cells,Vorinostat


In [25]:
test__ = id_map.copy()
test__.head()

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)
2,2,B cells,ABT737
3,3,B cells,AMD-070 (hydrochloride)
4,4,B cells,AT 7867


In [26]:
test__ = pd.merge(id_map,de_train[['SMILES','sm_name']].groupby('sm_name').max().reset_index(),on= 'sm_name',how= 'left')
test__.head()

,id,cell_type,sm_name,SMILES
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C
1,1,B cells,ABT-199 (GDC-0199),CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...
2,2,B cells,ABT737,CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...
3,3,B cells,AMD-070 (hydrochloride),NCCCCN(Cc1nc2ccccc2[nH]1)[C@H]1CCCc2cccnc21
4,4,B cells,AT 7867,Clc1ccc(C2(c3ccc(-c4cn[nH]c4)cc3)CCNCC2)cc1


In [27]:
test_df, feature_cols = test_data_creation(df = test__,label_encoder = label_encoder)

test_df

,id,cell_type,sm_name,SMILES,MYC,TP53,MLXIP,FOXO3,STAT1,ZBTB4,...,macss_157,macss_158,macss_159,macss_160,macss_161,macss_162,macss_163,macss_164,macss_165,macss_166
0,0,0,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
1,1,0,ABT-199 (GDC-0199),CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,2,0,ABT737,CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
3,3,0,AMD-070 (hydrochloride),NCCCCN(Cc1nc2ccccc2[nH]1)[C@H]1CCCc2cccnc21,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,1,0
4,4,0,AT 7867,Clc1ccc(C2(c3ccc(-c4cn[nH]c4)cc3)CCNCC2)cc1,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,1,Vandetanib,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
251,251,1,Vanoxerine,Fc1ccc(C(OCCN2CCN(CCCc3ccccc3)CC2)c2ccc(F)cc2)cc1,0,0,0,0,0,0,...,1,1,0,0,1,1,1,1,1,0
252,252,1,Vardenafil,CCCc1nc(C)c2c(=O)[nH]c(-c3cc(S(=O)(=O)N4CCN(CC...,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
253,253,1,Vorinostat,O=C(CCCCCCC(=O)Nc1ccccc1)NO,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0


Load Test data

In [28]:
import pandas as pd

# Load your dataset into a DataFrame
test_df = test_df

# Identify and remove duplicate columns
test_df = test_df.loc[:, ~test_df.columns.duplicated(keep='first')]

test_df

,id,cell_type,sm_name,SMILES,MYC,TP53,MLXIP,FOXO3,STAT1,ZBTB4,...,macss_157,macss_158,macss_159,macss_160,macss_161,macss_162,macss_163,macss_164,macss_165,macss_166
0,0,0,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
1,1,0,ABT-199 (GDC-0199),CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,2,0,ABT737,CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
3,3,0,AMD-070 (hydrochloride),NCCCCN(Cc1nc2ccccc2[nH]1)[C@H]1CCCc2cccnc21,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,1,0
4,4,0,AT 7867,Clc1ccc(C2(c3ccc(-c4cn[nH]c4)cc3)CCNCC2)cc1,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,1,Vandetanib,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
251,251,1,Vanoxerine,Fc1ccc(C(OCCN2CCN(CCCc3ccccc3)CC2)c2ccc(F)cc2)cc1,0,0,0,0,0,0,...,1,1,0,0,1,1,1,1,1,0
252,252,1,Vardenafil,CCCc1nc(C)c2c(=O)[nH]c(-c3cc(S(=O)(=O)N4CCN(CC...,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
253,253,1,Vorinostat,O=C(CCCCCCC(=O)Nc1ccccc1)NO,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0


In [29]:
svd = TruncatedSVD(n_components=n_components, random_state=1)
selected_feature_names = feature_cols
z_tr = svd.fit_transform(train_df[genes])
model.fit(train_df[selected_feature_names], z_tr)
y_pred = svd.inverse_transform(model.predict(test_df[selected_feature_names]))

In [30]:
submission = pd.DataFrame(y_pred, columns=genes, index=id_map.index)
submission.index.name = 'id'
display(submission)

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.378911,0.282552,0.639735,0.931955,0.857437,0.503366,-0.103986,0.578039,-0.328073,0.566302,...,-0.368355,0.115790,0.190373,0.130667,0.711308,0.504198,0.590307,0.483822,-0.230192,0.140803
1,0.071736,0.111922,0.098178,0.035713,0.135162,0.407701,0.024261,0.138319,0.034547,-0.112514,...,0.006041,0.066302,0.050657,0.212023,0.115864,0.111892,0.038708,0.015327,-0.028099,-0.028523
2,0.386882,0.256147,0.219281,0.039041,0.483975,1.316222,0.050409,0.310506,0.017086,-0.123977,...,-0.015743,0.149139,0.162676,0.361276,0.421134,0.370514,0.160785,0.069262,-0.049442,-0.093917
3,-0.001763,0.039465,0.123872,0.102244,0.128680,-0.135326,-0.058231,0.155926,-0.013928,0.129721,...,0.033528,0.048123,-0.071186,0.082959,0.172205,0.096325,0.143360,0.132359,0.009774,0.028615
4,0.181542,0.011958,0.003582,0.117694,0.337881,0.353005,-0.099487,0.157034,-0.094918,0.257153,...,-0.182819,0.035202,0.023169,0.038593,0.211379,0.140227,0.172464,0.145827,0.098752,-0.064582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,-0.179448,-0.147361,-0.195648,-0.184212,-0.318721,-0.397505,-0.107141,-0.129404,-0.002850,0.262588,...,-0.063924,-0.153278,-0.204638,-0.255529,-0.194776,-0.144273,-0.005679,0.037766,0.015853,-0.031038
251,0.075243,0.014125,0.096420,0.185741,0.121470,0.243124,-0.065143,0.052540,-0.146809,0.321683,...,-0.141256,-0.016008,0.017043,-0.009145,0.139271,0.118555,0.176591,0.156758,0.073457,-0.068369
252,-0.057414,-0.032274,-0.023957,0.008986,-0.068584,-0.107249,-0.055559,0.063045,-0.027214,0.214796,...,-0.034597,-0.009495,-0.058766,-0.049636,0.032744,-0.005776,0.105556,0.113963,0.024843,0.020275


In [31]:
submission.to_csv('submission_wb.csv')

In [32]:
#group by drug and take the mean
df_simple = de_train.iloc[:, [1] + list(range(5, de_train.shape[1]))]

mean_df_aggr = df_simple.groupby('sm_name').mean().reset_index()

mean_df_aggr.shape

(146, 18212)

In [33]:
df_submit_aggr_compund = pd.merge(id_map, mean_df_aggr, on = 'sm_name', how = 'left').sort_values('id').drop(columns = ['cell_type', "sm_name"]).set_index('id')

df_submit_aggr_compund.to_csv("comp.csv")

In [34]:
df_simple

,sm_name,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,Clotrimazole,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,Clotrimazole,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,Clotrimazole,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,Clotrimazole,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,Mometasone Furoate,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,Atorvastatin,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,-0.544709,0.282458,-0.431359,-0.364961,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,Riociguat,-0.455549,0.188181,0.595734,-0.100299,0.786192,0.090954,0.169523,0.428297,0.106553,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,Riociguat,0.338168,-0.109079,0.270182,-0.436586,-0.069476,-0.061539,0.002818,-0.027167,-0.383696,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,Riociguat,0.101138,-0.409724,-0.606292,-0.071300,-0.001789,-0.706087,-0.620919,-1.485381,0.059303,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


In [35]:
#group by drug and take the mean
# df_simple_2 = de_train.iloc[:, [0] + list(range(5, de_train.shape[1]))]
df_simple_2 = de_train.iloc[:, [0] + list(range(5, de_train.shape[1]))]


mean_cell_type = df_simple_2.groupby('cell_type').mean().reset_index()

mean_cell_type.shape

(6, 18212)

In [36]:
df_submit_cell_type = pd.merge(id_map, mean_cell_type, on = 'cell_type', how = 'left').sort_values('id').drop(columns = ['cell_type', "sm_name"]).set_index('id')

df_submit_cell_type.to_csv('cell_type.csv')

In [37]:
df_submit_cell_type

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,1.380890,0.530585,1.340812,1.594307,4.927551,3.613191,0.028640,0.544636,0.723079,0.541773,...,0.257778,0.674977,0.217386,1.439374,0.952903,0.581303,0.637408,0.517737,-0.207092,0.079199
1,1.380890,0.530585,1.340812,1.594307,4.927551,3.613191,0.028640,0.544636,0.723079,0.541773,...,0.257778,0.674977,0.217386,1.439374,0.952903,0.581303,0.637408,0.517737,-0.207092,0.079199
2,1.380890,0.530585,1.340812,1.594307,4.927551,3.613191,0.028640,0.544636,0.723079,0.541773,...,0.257778,0.674977,0.217386,1.439374,0.952903,0.581303,0.637408,0.517737,-0.207092,0.079199
3,1.380890,0.530585,1.340812,1.594307,4.927551,3.613191,0.028640,0.544636,0.723079,0.541773,...,0.257778,0.674977,0.217386,1.439374,0.952903,0.581303,0.637408,0.517737,-0.207092,0.079199
4,1.380890,0.530585,1.340812,1.594307,4.927551,3.613191,0.028640,0.544636,0.723079,0.541773,...,0.257778,0.674977,0.217386,1.439374,0.952903,0.581303,0.637408,0.517737,-0.207092,0.079199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,1.570336,0.752564,-2.856826,0.887845,6.658911,4.034911,0.442943,0.403543,0.196285,0.710763,...,-0.270423,-0.103318,-1.307952,-0.166312,1.883588,0.612681,-0.583563,-0.427938,-0.292768,-0.067723
251,1.570336,0.752564,-2.856826,0.887845,6.658911,4.034911,0.442943,0.403543,0.196285,0.710763,...,-0.270423,-0.103318,-1.307952,-0.166312,1.883588,0.612681,-0.583563,-0.427938,-0.292768,-0.067723
252,1.570336,0.752564,-2.856826,0.887845,6.658911,4.034911,0.442943,0.403543,0.196285,0.710763,...,-0.270423,-0.103318,-1.307952,-0.166312,1.883588,0.612681,-0.583563,-0.427938,-0.292768,-0.067723


In [38]:
submit_df_aggr_comp = pd.read_csv(r'/kaggle/working/comp.csv')
submit_df_aggr_comp.drop('id',axis=1,inplace=True)
submit_df_celltype = pd.read_csv(r'/kaggle/working/cell_type.csv')
submit_df_celltype.drop('id',axis=1,inplace=True)

In [39]:
# Take the average
average_df = submission*0.40 + df_submit_aggr_compund*0.40 + submit_df_celltype*0.1
average_df.head()

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.409760,0.121106,0.555233,1.119666,1.129025,0.851650,0.011413,0.370041,-0.409625,0.268473,...,-0.429395,-0.162195,0.219334,0.300478,0.601925,0.594960,0.477678,0.418268,-0.200729,0.285004
1,0.134269,0.100753,0.205849,0.123405,0.634608,0.627715,-0.051659,0.119350,0.213115,-0.032358,...,0.200509,0.131957,0.006543,0.172543,0.228165,0.076510,0.124533,-0.014793,-0.004711,-0.040781
2,0.456046,0.284547,0.264773,0.155378,0.855259,1.348417,0.323772,0.254447,0.030684,0.036539,...,0.094098,0.055133,0.098082,0.453854,0.330535,0.337371,0.230675,0.051627,-0.094873,-0.320857
3,0.124932,0.282304,0.233525,0.296922,0.537125,0.323048,-0.090014,0.439634,0.058899,0.200490,...,-0.001147,0.112875,0.070470,0.301447,0.171695,0.154427,0.121233,0.186508,-0.047928,0.085902
4,0.307800,-0.052494,0.198838,0.313455,0.626569,0.575942,-0.128247,0.182195,-0.062003,0.274328,...,-0.329224,0.046058,-0.017212,0.024382,0.200336,0.300855,0.069836,0.359156,0.027752,0.013774


In [40]:
average_df.to_csv('submission_07.csv')

In [42]:
average_df

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.409760,0.121106,0.555233,1.119666,1.129025,0.851650,0.011413,0.370041,-0.409625,0.268473,...,-0.429395,-0.162195,0.219334,0.300478,0.601925,0.594960,0.477678,0.418268,-0.200729,0.285004
1,0.134269,0.100753,0.205849,0.123405,0.634608,0.627715,-0.051659,0.119350,0.213115,-0.032358,...,0.200509,0.131957,0.006543,0.172543,0.228165,0.076510,0.124533,-0.014793,-0.004711,-0.040781
2,0.456046,0.284547,0.264773,0.155378,0.855259,1.348417,0.323772,0.254447,0.030684,0.036539,...,0.094098,0.055133,0.098082,0.453854,0.330535,0.337371,0.230675,0.051627,-0.094873,-0.320857
3,0.124932,0.282304,0.233525,0.296922,0.537125,0.323048,-0.090014,0.439634,0.058899,0.200490,...,-0.001147,0.112875,0.070470,0.301447,0.171695,0.154427,0.121233,0.186508,-0.047928,0.085902
4,0.307800,-0.052494,0.198838,0.313455,0.626569,0.575942,-0.128247,0.182195,-0.062003,0.274328,...,-0.329224,0.046058,-0.017212,0.024382,0.200336,0.300855,0.069836,0.359156,0.027752,0.013774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0.082824,-0.252787,-0.456077,-0.182079,0.582173,0.032896,0.013612,0.050616,-0.033604,0.408982,...,-0.007185,-0.094386,-0.362926,-0.158314,0.029150,-0.121783,-0.018350,-0.029603,0.031185,0.082431
251,0.262331,-0.102948,-0.204147,0.340133,0.829942,0.596591,-0.058230,-0.041693,-0.098719,0.283601,...,-0.074257,-0.062695,-0.064631,0.046238,0.332200,0.070372,-0.064231,0.163294,-0.098682,-0.122671
252,0.095140,0.273065,-0.430941,0.029741,0.647091,0.252901,-0.064574,-0.113613,0.027911,0.306203,...,-0.023262,0.006064,0.057239,0.028697,0.233306,0.022279,-0.078129,0.079514,0.110782,-0.152790


Final prediction on test data

In [43]:
d = pd.read_csv("/kaggle/working/submission_07.csv")
d

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.409760,0.121106,0.555233,1.119666,1.129025,0.851650,0.011413,0.370041,-0.409625,...,-0.429395,-0.162195,0.219334,0.300478,0.601925,0.594960,0.477678,0.418268,-0.200729,0.285004
1,1,0.134269,0.100753,0.205849,0.123405,0.634608,0.627715,-0.051659,0.119350,0.213115,...,0.200509,0.131957,0.006543,0.172543,0.228165,0.076510,0.124533,-0.014793,-0.004711,-0.040781
2,2,0.456046,0.284547,0.264773,0.155378,0.855259,1.348417,0.323772,0.254447,0.030684,...,0.094098,0.055133,0.098082,0.453854,0.330535,0.337371,0.230675,0.051627,-0.094873,-0.320857
3,3,0.124932,0.282304,0.233525,0.296922,0.537125,0.323048,-0.090014,0.439634,0.058899,...,-0.001147,0.112875,0.070470,0.301447,0.171695,0.154427,0.121233,0.186508,-0.047928,0.085902
4,4,0.307800,-0.052494,0.198838,0.313455,0.626569,0.575942,-0.128247,0.182195,-0.062003,...,-0.329224,0.046058,-0.017212,0.024382,0.200336,0.300855,0.069836,0.359156,0.027752,0.013774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.082824,-0.252787,-0.456077,-0.182079,0.582173,0.032896,0.013612,0.050616,-0.033604,...,-0.007185,-0.094386,-0.362926,-0.158314,0.029150,-0.121783,-0.018350,-0.029603,0.031185,0.082431
251,251,0.262331,-0.102948,-0.204147,0.340133,0.829942,0.596591,-0.058230,-0.041693,-0.098719,...,-0.074257,-0.062695,-0.064631,0.046238,0.332200,0.070372,-0.064231,0.163294,-0.098682,-0.122671
252,252,0.095140,0.273065,-0.430941,0.029741,0.647091,0.252901,-0.064574,-0.113613,0.027911,...,-0.023262,0.006064,0.057239,0.028697,0.233306,0.022279,-0.078129,0.079514,0.110782,-0.152790
253,253,0.250221,0.198487,-0.033322,0.025660,0.834376,0.254948,0.427137,0.590279,1.129686,...,0.197034,0.509635,-0.352521,-0.069349,0.008351,0.359783,-0.047982,0.202611,0.059020,0.051495
